In [634]:
from dotenv import load_dotenv
import os
from pathlib import Path
import psycopg2

import pandas as pd

import plotly.express as px

In [598]:
# Getting the database variables from the .env file
load_dotenv(dotenv_path=Path('../path/to/.env'))

DB_KEY = os.getenv("DB_KEY")
DB_NAME = os.getenv("DB_NAME")

# Connecting to the database
conn = psycopg2.connect(
    dbname='PEDE_PASSOS',
    user=DB_NAME,
    host='localhost',
    port='5432',
    password=DB_KEY)

In [599]:
# Reading the data from the database
data = pd.read_sql("SELECT * FROM pede_passos.students_grade_comp", con=conn)

db_data_path = Path('../data/interim/students_grades.csv')

data.to_csv(db_data_path, index=False)

/var/folders/s8/p68_rlhj54bb1q_xvzdsc1m40000gn/T/ipykernel_85335/2732130830.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql("SELECT * FROM pede_passos.students_grade_comp", con=conn)


In [600]:
treated_data = data.replace(['INCLUIR', '#N/A', '#DIV/0!'], None)

In [601]:
treated_data = treated_data.dropna(thresh=len(treated_data.columns) - 2)

In [602]:
columns_to_adjust = ['INDE', 'IAA', 'IEG', 'IPS', 'IDA', 'IPP', 'IPV', 'IAN']

treated_data[columns_to_adjust] = treated_data[columns_to_adjust].apply(lambda x: x.str.replace(',', '.'))

In [603]:
treated_data[columns_to_adjust] = treated_data[columns_to_adjust].apply(pd.to_numeric, errors='coerce')

In [604]:
treated_data.loc[treated_data['ANO'] == 2024, 'FASE'].unique()

array(['ALFA', '1A', '1B', '1C', '1D', '1E', '1G', '1H', '1J', '1K', '1L',
       '1M', '1N', '1P', '1R', '2A', '2B', '2C', '2D', '2G', '2H', '2I',
       '2K', '2L', '2M', '2N', '2P', '2R', '2U', '3A', '3B', '3C', '3D',
       '3F', '3G', '3H', '3I', '3K', '3L', '3M', '3N', '3P', '3R', '3U',
       '4A', '4B', '4C', '4F', '4H', '4L', '4M', '4N', '4R', '5A', '5B',
       '5C', '5D', '5F', '5G', '5L', '5M', '5N', '6A', '6L', '7A', '7E'],
      dtype=object)

In [605]:
treated_data.loc[treated_data['ANO'] == 2020, 'FASE'] = treated_data.loc[treated_data['ANO'] == 2020, 'FASE'].str[0]
treated_data.loc[treated_data['ANO'] == 2021, 'FASE'] = treated_data.loc[treated_data['ANO'] == 2021, 'FASE'].str[0]
treated_data.loc[treated_data['ANO'] == 2023, 'FASE'] = treated_data.loc[treated_data['ANO'] == 2023, 'FASE'].str[-1]
treated_data.loc[treated_data['ANO'] == 2023, 'FASE'] = treated_data.loc[treated_data['ANO'] == 2023, 'FASE'].str.replace('A', '0')
treated_data.loc[treated_data['ANO'] == 2024, 'FASE'] = treated_data.loc[treated_data['ANO'] == 2024, 'FASE'].str[0]
treated_data.loc[treated_data['ANO'] == 2024, 'FASE'] = treated_data.loc[treated_data['ANO'] == 2024, 'FASE'].str.replace('A', '0')


In [606]:
treated_data['FASE'] = treated_data['FASE'].apply(pd.to_numeric, errors='coerce').astype('Int64')

In [607]:
gender_dict = {'Masculino': 'M', 'Feminino': 'F',
               'Menino': 'M', 'Menina': 'F',
               'M': 'M', 'F': 'F'}

treated_data['GENERO'] = treated_data['GENERO'].map(gender_dict)

In [608]:
def corrigir_idade(idade):
    try:
        # Tenta converter para data
        data_to_adjust = pd.to_datetime(idade, errors='coerce', dayfirst=False)  # Formato MM/DD/YYYY
        if pd.notna(data_to_adjust):
            return data_to_adjust.day  # Retorna apenas o dia
        return int(idade)  # Se já for um número, mantém como está
    except:
        return None

In [609]:
treated_data['IDADE'] = treated_data['IDADE'].apply(corrigir_idade)

In [615]:
treated_data['ENSINO'].groupby(treated_data['ENSINO']).count().sort_values(ascending=False)

ENSINO
Particular    2531
Pública       1714
Name: ENSINO, dtype: int64

In [614]:
not_defined_school = ['Nenhuma das opções acima', 'Concluiu o 3º EM', 'Pública']

treated_data['ENSINO'] = treated_data['ENSINO'].str.replace('Escola Pública', '')
treated_data['ENSINO'].loc[~treated_data['ENSINO'].isin(not_defined_school)] = 'Particular'
treated_data['ENSINO'].loc[treated_data['ENSINO'].isin(not_defined_school[:-1])] = pd.NA


/var/folders/s8/p68_rlhj54bb1q_xvzdsc1m40000gn/T/ipykernel_85335/3768940558.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  treated_data['ENSINO'].loc[~treated_data['ENSINO'].isin(not_defined_school)] = 'Particular'
/var/folders/s8/p68_rlhj54bb1q_xvzdsc1m40000gn/T/ipykernel_85335/3768940558.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  treated_data['ENSINO'].loc[treated_data['ENSINO'].isin(not_defined_school[:-1])] = pd.NA


In [619]:
treated_data['NOME'] = treated_data['NOME'].str.upper()
treated_data.reset_index(drop=True, inplace=True)

In [625]:
inde_corr = treated_data.corr(numeric_only=True)['INDE'].sort_values(ascending=False)

In [ ]:
# Contar o número de alunos por ano
alunos_por_ano = treated_data.groupby('ANO').size().reset_index(name='Count')

# Fazer o plot
fig = px.bar(alunos_por_ano, x='ANO', y='Count', title='Número de Alunos por Ano', template='ggplot2')
fig.update_layout(xaxis_title='Ano', yaxis_title='Número de Alunos', width=1000, height=500)
fig.update_traces(texttemplate='%{y}', textposition='outside')
fig.show()